In [1]:
import pandas as pd
import os

# Chemin du fichier
CHEMIN_FICHIER = "Z:/forgetting/Carla/OpenEphysData/Violet_Rouge_Vert_2025-02-28_17-57-01_Sleep/Record Node 105/experiment2/recording1/continuous/Acquisition_Board-103.Rhythm Data/Sleep_ScoringViolet_4Stages_5sEpoch.csv"

def load_data(file_path):
    """Charge les données depuis un fichier Excel ou CSV."""
    if file_path.endswith(('.xlsx', '.xls')):
        df = pd.read_excel(file_path)
    elif file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    else:
        # Essayer d'abord comme CSV, puis comme Excel si cela échoue
        try:
            df = pd.read_csv(file_path)
        except:
            df = pd.read_excel(file_path)
    
    return df

def calculate_grouped_periods(df):
    """Calcule les périodes regroupées pour chaque type de label."""
    results = []
    current_label = None
    start_time = None
    total_duration = 0
    
    for i, row in df.iterrows():
        # Extraire le type de label (REM, NREM, Wake) sans le numéro
        label_type = row['label'].split('_')[0]
        
        # Si c'est le premier enregistrement ou si le type a changé
        if current_label is None or label_type != current_label:
            # Si ce n'est pas le premier enregistrement, ajouter la période précédente aux résultats
            if current_label is not None:
                results.append({
                    'start_time': start_time,
                    'end_time': start_time + total_duration,
                    'duration': total_duration,
                    'label': current_label
                })
            
            # Réinitialiser pour la nouvelle période
            current_label = label_type
            start_time = row['time']
            total_duration = row['duration']
        else:
            # Continuer la période actuelle
            total_duration += row['duration']
    
    # Ajouter la dernière période
    if current_label is not None:
        results.append({
            'start_time': start_time,
            'end_time': start_time + total_duration,
            'duration': total_duration,
            'label': current_label
        })
    
    return pd.DataFrame(results)

def main():
    file_path = CHEMIN_FICHIER
    
    # Vérifier si le fichier existe
    if not os.path.exists(file_path):
        print(f"Le fichier '{file_path}' n'existe pas. Veuillez corriger le chemin.")
        return
    
    # Charger les données
    try:
        df = load_data(file_path)
        print(f"Fichier chargé avec succès: {file_path}")
        print(f"Aperçu des données:\n{df.head()}")
    except Exception as e:
        print(f"Erreur lors du chargement du fichier: {e}")
        return
    
    # Calculer les périodes regroupées
    grouped_periods = calculate_grouped_periods(df)
    
    # Afficher les résultats
    print("\nPériodes regroupées :")
    print(grouped_periods)
    
    # Calculer le nombre de périodes par type de label
    label_counts = grouped_periods['label'].value_counts()
    print("\nNombre de périodes par type de label :")
    print(label_counts)
    
    # Calculer la durée totale pour chaque type de label
    label_durations = grouped_periods.groupby('label')['duration'].sum()
    print("\nDurée totale par type de label :")
    print(label_durations)

    # Calculer la durée moyenne par type de label
    label_mean_duration = grouped_periods.groupby('label')['duration'].mean()
    print("\nDurée moyenne par type de label :")
    print(label_mean_duration)
    
    # Créer un DataFrame de résumé
    summary_df = pd.DataFrame({
        'Nombre de périodes': label_counts,
        'Durée totale (minutes)': label_durations,
        'Durée moyenne (minutes)': label_mean_duration
    }).reset_index().rename(columns={'index': 'Type de label'})
    
    # Enregistrer les résultats dans le même dossier que le fichier d'origine
    output_dir = os.path.dirname(file_path)
    base_filename = os.path.splitext(os.path.basename(file_path))[0]
    
    # Enregistrer le tableau des périodes regroupées
    periods_output_path = os.path.join(output_dir, f"{base_filename}_periodes_regroupees.xlsx")
    grouped_periods.to_excel(periods_output_path, index=False)
    print(f"\nPériodes regroupées enregistrées dans: {periods_output_path}")
    
    # Enregistrer le résumé
    summary_output_path = os.path.join(output_dir, f"{base_filename}_resume.xlsx")
    summary_df.to_excel(summary_output_path, index=False)
    print(f"Résumé enregistré dans: {summary_output_path}")

if __name__ == "__main__":
    main()


Fichier chargé avec succès: Z:/forgetting/Carla/OpenEphysData/Violet_Rouge_Vert_2025-02-28_17-57-01_Sleep/Record Node 105/experiment2/recording1/continuous/Acquisition_Board-103.Rhythm Data/Sleep_ScoringViolet_4Stages_5sEpoch.csv
Aperçu des données:
   time  duration  label
0   0.0       5.0  REM_3
1   5.0       5.0  REM_3
2  10.0       5.0  REM_3
3  15.0       5.0  REM_3
4  20.0       5.0  REM_3

Périodes regroupées :
       start_time      end_time  duration label
0        0.000000     47.900000      47.9   REM
1       47.900000     68.000000      20.1  Wake
2       68.000000    142.000000      74.0  NREM
3      142.000000    147.000000       5.0  Wake
4      147.000000    533.000000     386.0  NREM
..            ...           ...       ...   ...
405  76264.455648  76288.855648      24.4  Wake
406  76288.855648  76318.855648      30.0  NREM
407  76318.855648  76410.755648      91.9   REM
408  76410.755648  76490.755648      80.0  Wake
409  76490.755648  76670.755648     180.0  NREM

